In [8]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

key_training_data = []
chord_training_data = []

class Labeling():        
        
    def make_training_data():
        
        
        keys = ['C','C#','Db','D','D#','Eb','E','F','F#','Gb','G','G#','Ab','A','A#','Bb','B']
        keyIndex = [0, 1, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11]
        chords = ['minor', 'major']
        chordsIndex = [0, 1]
        chordsCounters = [0, 0]
        path = "C:/outs/spects4096"
        os.chdir(path)
        spectrograms = os.listdir()
        key_count = [0] * 24
        annotations = open("C:/outs/shiftedAnnotations4096.txt", "r")
        while True: 
            line = annotations.readline()
            if not line:
                break;
            
            file, key, chord = line.split(' ', 2)
            key = key.strip()
            chord = chord.strip()
            file = file + ".LOFI.png"
            spect_path = os.path.join(path, file)
            #spect_path = cv2.imread(spect_path)
            spect_path = cv2.imread(spect_path, cv2.IMREAD_GRAYSCALE)
            
            if spect_path is None :
                continue;
                
            
            for j,i in enumerate(chords):
                if (chordsCounters[0] > 100) & (chord == 'minor'):
                    continue;
                    
                if (chordsCounters[1] > 100) :
                    continue;
                    
                if chord == i:
                    chord_training_data.append([np.array(spect_path)/255, torch.LongTensor([chordsIndex[j]])])
                    chordsCounters[j] += 1

        
        
        np.random.shuffle(chord_training_data)
        #print("saving data.....ornah")
        #np.save("chord_training_data.npy", chord_training_data)
        
        print(chordsCounters)
              
if __name__ == '__main__':
    Labeling.make_training_data()  
    

[101, 101]


In [9]:
#CHORD ESTIMATION NEURAL NETWORK (binary classification)"

import torch
import torch.nn as nn
import torch.nn.functional as F


class binaryNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,16,5)
        self.conv2 = nn.Conv2d(16,16,5)
        
        x = torch.randn(105,600).view(-1,1,105,600) # giving a random input to find the input of the first linear layer 
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 8)
        self.fc2 = nn.Linear(8, 2)
        self.dropout = nn.Dropout(0.5)        
        
        
    def convs(self, x):
        x = F.elu(self.conv1(x))
        
        x = F.avg_pool2d(F.elu(self.conv2(x)), (2,2))
        
        if self._to_linear is None: # define the input of the first linear layer 
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    
    def forward(self, x):
        
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.elu(self.fc1(x))
        x = self.dropout(x)
        x = F.elu(self.fc2(x))
        
        return x
        
        
if torch.cuda.is_available(): # set values to GPU
    device = torch.device("cuda:0")
    c_Net = binaryNet().to(device)


print(c_Net)




binaryNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=227328, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [10]:
from tqdm import tqdm
import torch.optim as optim


optimizer = torch.optim.SGD(c_Net.parameters(), lr=0.0005, momentum = 0.9, weight_decay = 0.0001) 
loss_function = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    loss_function = loss_function.to(device)

print("optimizer komple ")

optimizer komple 


In [15]:
#TRAINING 
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt



val_size = int(len(chord_training_data)*0.1)

training_data = chord_training_data[:len(chord_training_data)-val_size]


batchSize = 8
train_dataloader = DataLoader(training_data, batch_size=batchSize , shuffle=True, pin_memory=True, drop_last=True)


num_epochs = 10
test_counter = 0
for epoch in range(0, num_epochs):
    running_loss = 0.0
    loss_values = [] 
    
    for i, data in enumerate(train_dataloader, 0):
        test_counter += 1
        inputs = data[0].view(-1,1,105,600) # set the inputs that will be given to neural network
        targets = data[1] # targets are the correct keys
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device) # set variables to GPU

        optimizer.zero_grad()
        outputs = c_Net(inputs) # generate predictions 
        
        targets = targets.view(len(data[1]))
        
        loss = loss_function(outputs, targets) 
        loss.backward() # backpropagation
        optimizer.step()
        running_loss =+ loss.item() * batchSize
        loss_values.append(running_loss / len(training_data))
        
    print(f"Epoch: {epoch}. Loss: {loss}") 
            

#plt.plot(loss_values)        

Epoch: 0. Loss: 0.3403049409389496
Epoch: 1. Loss: 0.1418224573135376
Epoch: 2. Loss: 0.38320043683052063
Epoch: 3. Loss: 0.22406889498233795
Epoch: 4. Loss: 0.06610433757305145
Epoch: 5. Loss: 0.10050226747989655
Epoch: 6. Loss: 0.03841809183359146
Epoch: 7. Loss: 0.04669838026165962
Epoch: 8. Loss: 0.04279829561710358
Epoch: 9. Loss: 0.026922201737761497


In [16]:
#TESTING 
correct, total = 0, 0
batchSize = 8
test_data = chord_training_data[-val_size:]
test_dataloader = DataLoader(test_data, batch_size= batchSize, shuffle=True, drop_last=True)

c_Net.eval()

print("``````Starting Testing``````")
with torch.no_grad():
    running_loss = 0.0
    loss_values = [] 

    # Iterate over the test data and generate predictions
    for i, data in enumerate(test_dataloader, 0):
        # Get inputs
        inputs = data[0].view(-1,1,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)
        # Generate outputs
        outputs = c_Net(inputs)
        
        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        running_loss =+ loss.item() * batchSize
        
        cntr = 0 # enumarate ?!?
        for i in predicted:

            if i == targets[cntr]:
                correct += 1
            cntr += 1

        loss_values.append(running_loss / len(test_data))
    
print("Accuracy: ", 100*round(correct/total, 3))

#plt.plot(loss_values)

``````Starting Testing``````
Accuracy:  100.0


In [ ]:
#torch.save(c_Net.state_dict(), 'C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/saved_nns/best-chord-network-model-parameters.pt')


In [ ]:
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X_chords)*VAL_PCT)


#train_keys_X = X_keys[:len(key_training_data)-val_size]
#train_keys_y = y_keys[:len(key_training_data)-val_size]
                                                                            # 2 train/test gia kathe net
train_chords_X = X_chords[:len(chord_training_data)-val_size]
train_chords_y = y_chords[:len(chord_training_data)-val_size]

print(train_chords_X)

#test_keys_X = X_keys[-val_size:]
#test_keys_y = y_keys[-val_size:]

test_chords_X = X_chords[-val_size:]
test_chords_y = y_chords[-val_size:]

In [ ]:
#****KEY TRAIN*****

BATCH_SIZE = 16
EPOCHS = 30

def K_train(net):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_keys_X), BATCH_SIZE)):
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_keys_X[i:i+BATCH_SIZE].view(-1, 1, 52, 300)
            batch_y = train_keys_y[i:i+BATCH_SIZE]#```````````````````````````````````````````````mono auta allazeis 
            
           #batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()

            outputs = net(batch_X)
            
            #print(outputs.shape)
            batch_y = batch_y.long()
            #print(batch_y.shape)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")

In [ ]:
#*****CHORD TRAIN******

import matplotlib.pyplot as plt

BATCH_SIZE = 32
EPOCHS = 10

def C_train(c_net):
    for epoch in range(EPOCHS):
        running_loss = 0.0
        loss_values = [] 
        for i in tqdm(range(0, len(train_chords_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_chords_X[i:i+BATCH_SIZE].view(-1, 1, 105, 600)
            batch_y = train_chords_y[i:i+BATCH_SIZE]   #``````````````````````````````````````````````````````````````````````````````````````````````theloume 2 train ksexwrista alliws ta kanoume kai mazi
            
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            c_net.zero_grad()

            outputs = c_net(batch_X)
            
            #print(outputs.shape)
            batch_y = batch_y.long()
            #print(batch_y.shape)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update
            running_loss =+ loss.item() * batch_X.size(0)
            
            
        loss_values.append(running_loss / len(train_chords_X))
            
        print(f"Epoch: {epoch}. Loss: {loss}")
        plt.plot(loss_values)
        
        

In [ ]:
C_train(c_net)

In [ ]:
#******CHORD TEST****

correct = 0
total = 0

#test_X , test_y = test_X.to(device), test_y.to(device)
with torch.no_grad():
    loss_values = []
    running_loss = 0.0
    for i in tqdm(range(len(test_X))):
        #real_class = torch.argmax(test_y[i])
        net_out = c_net(test_chords_X[i].view(-1, 1, 105, 600))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)
        running_loss =+ loss.item() * batch_X.size(0)
        
        #print(predicted_class, test_y[i])
        if predicted_class == test_chords_y[i]:
            correct += 1
        total += 1

        
        
loss_values.append(running_loss / len(train_chords_X))
    
print("Accuracy: ", round(correct/total, 3))

plt.plot(loss_values)